In [49]:
import argparse
import io
import json
import os

from google.cloud import language
import numpy
import six

In [ ]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [ ]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=12dd834544b38eca27f8853a94af0eb2b65d392844719191ca518c3447d4ea13
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [ ]:
language_client = language.LanguageServiceClient()

def classify(text, verbose=True):
    """Classify the input text into categories. """
    document = language.types.Document(
        content=text,
        type=language.enums.Document.Type.PLAIN_TEXT)
    response = language_client.analyze_entities(document)
    return response

import requests
import wikipedia
def findWikiData(itemurl):
    st = itemurl.find('wiki/')
    item = itemurl[st+5:]
    #print(item)
    s= "https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&titles=%s&format=json"%(item)
    r = requests.get(s)
    rr =r.content.decode("utf-8")
    #print('rr=', rr)
    ans = rr.find('wikibase_item')
    answer = rr[ans+16:]
    e = answer.find('"')
    return item, answer[:e]

In [ ]:
def getInstanceOfId(wikiID):
    s = 'https://www.wikidata.org/w/api.php?action=wbgetentities&ids=%s&languages=en&format=json'%(wikiID)
    r = requests.get(s)
    rr =r.content.decode("utf-8")
    try:
        j = json.loads(rr)
        k = rr.find('P31')
        if k > 0:
            l = rr[k:].find('"id":')
            if l > 0:
                s = rr[k+l+5:k+l+23]
                t = s.find('}')
                id = s[1:t-1]
                x = id.find('$')
                if x < 0:
                    return id
                else:
                    return id[:x]
            else:
                return ''
        else:
            return ''
    except:
        return ''
    
    

In [ ]:
def getNameFromWikiID(wikiID):
    s = 'https://www.wikidata.org/w/api.php?action=wbgetentities&ids=%s&languages=en&format=json'%(wikiID)
    r = requests.get(s)
    if r.status_code == 429:
        time.sleep(8)
        r = requests.get(s)
    rr = r.content.decode("utf-8")
    s = rr.find('"value":')
    rrr= rr[s+8:s+50]
    t = rrr.find('}')
    return rrr[1:t-1]

In [ ]:
def getDescription(wikiId):
    s = 'https://www.wikidata.org/w/api.php?action=wbgetentities&ids=%s&languages=en&format=json'%(wikiId)
    r = requests.get(s)
    if r.status_code == 429:
      time.sleep(8)
      r = requests.get(s)

    rr =r.content.decode("utf-8")
    i = rr.find("descriptions")
    if i < 0:
        return ''
    else:
        rrr = rr[i:]
        #print(rrr)
        j = rrr.find('"value":')
        if j >0:
            #print(j)
            k = rrr.find("}")
            return(rrr[j+8: k])
        else:
            return ''


In [ ]:
def entityExtractorBlock(sents):
    ents = []
    x = classify(sents)
    for ent in x.entities:
        #print('--->', ent.name)
        if ent.metadata['wikipedia_url'] != '' or len(str.split(ent.name))> 1:
            wikipage = ent.metadata['wikipedia_url']
            if wikipage != '':
                name, wikidataId = findWikiData(wikipage)
                ents.append((name, wikidataId, wikipage))

            else:
                wikidataId = "none"
                #print(name, wikidataId, wikipage)
                ents.append((ent.name, wikidataId, wikipage))
        
    return ents

In [ ]:
import networkx as nx

In [ ]:
def entityExtractor(sent):
    ents = []
    newents = entityExtractorBlock(sent)
    if newents != []:
        entitem = []
        for x in newents:
            entitem.append({'entity': x})
        entitem.append({'context': sent})
        ents = ents + entitem
    #print(entitem)
    return ents

In [ ]:
def addFile(art_cnt, G, filename):
    items = open('../'+filename+'.txt', 'r').read().replace('\n', '')
    sents = split_into_sentences(items)
    for sent in sents:
        #print(sent)
        ents = entityExtractor(sent)
        all_nodes = []
        nodes = []
        instances = []
        ent_list = []
        for e in ents:
            if e.get('entity') != None:
                ent_list.append(e['entity'][0])
                #this is an entity for this node
                lab = e['entity'][2]
                name = e['entity'][0]
                wikiID = e['entity'][1]
                st = lab.find('wiki/')
                if st <0:
                    item = name
                    #print('Item=', item)
                    G.add_node(item, flavor='entity', url='none', wikiID = 'noID', color = 'lightgreen')
                    #print("just made ", G.node[item])
                    G.nodes[item]['instanceof'] = []
                    G.nodes[item]['subclassof'] = []
                    G.nodes[item]['description'] = ''
                    nodes.append(item)

                else:
                    item = lab[st+5:]
                    #print("item=", item)
                    #wikiID = findWikiData(lab)[1]
                    G.add_node(item, flavor='entity', url=lab, wikiID = wikiID, color = 'lightblue')
                    #print("just smade ", item, G.nodes[item])
                    instID = getInstanceOfId(wikiID)
                    if instID != '':
                        inst_list = [(instID, getNameFromWikiID(instID))]
                        #print([wikiID, (instID, getNameFromWikiID(instID))])
                    else:
                        inst_list = []
                    clas_list = []#isASubclassOf(wikiID)
                    G.nodes[item]['instanceof'] = inst_list
                    instances = instances + inst_list
                    G.nodes[item]['subclassof'] = clas_list        
                    #classes = classes + clas_list
                    G.nodes[item]['description'] = getDescription(wikiID)
                    #print(findWikiData(lab))
                    nodes.append(item)
            if e.get('context') != None:
                # this is a new article
                newart = 'Art'+str(art_cnt)
                G.add_node(newart, flavor='article', source=filename, context=e['context'], color='lightyellow')
                G.nodes[newart]['ent-list'] = ent_list
                #print(newart,G.nodes[newart])
                art_cnt = art_cnt+1
                for z in nodes:
                    G.add_edge(newart,z )
                all_nodes = all_nodes + nodes
                nodes = []
        instance_set = set(instances)
        instances = list(instance_set)
        if instances != []:
            print('instances ====', instances)
        #now make items for these instances
        for i in instances:
            G.add_node(i[1], flavor='entity', url='notknown', wikiID = i[0], color = 'lightgray')
            G.nodes[i[1]]['description'] = getDescription(i[0])
            G.nodes[i[1]]['instanceof'] = []
            G.nodes[i[1]]['subclassof'] = []
        #now add edges from the instance to the new entity
        all_nodes_set = set(all_nodes)
        all_nodes =  list(all_nodes_set)
        for n in all_nodes:
            if G.nodes[n]['instanceof'] != []:
                elist = G.nodes[n]['instanceof']
                for e in elist:
                    G.add_edge(n, e[1])
    print(art_cnt)
    return art_cnt

In [ ]:
G = nx.read_gpickle('phy_plus_geo_singlesent')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def showGraph(sg):
    options = {
        'node_size': 3000,
        'font_weight': 'bold',
         'edgecolors':'black',
        'width': 2,
    }
    labels = {}
    color = []
    for n in sg.nodes:
        d =sg.nodes[n]
        #print(d)
        color.append(d['color'])
        labels[n]=n
    
    #print(color)
        #print(sg.nodes[x]['url'])
        #labels[]
        
    plt.figure(3,figsize=(15,15)) 

    nx.draw(sg, labels = labels, node_color = color, **options)

In [ ]:
def showArticle(graph, name):
    print(name, graph.nodes[name]['context'])
    print('all edges from this node are:')
    for x in graph.edges():
        if x[1] == name:
            print(x, graph.nodes[x[0]]['url'], graph.nodes[x[0]]['wikiID'])
            

def showEntity(graph, name):
    instances = []
    classes = []
    print(name,' wikiID=', graph.nodes[name]['wikiID'], 'wikipedia url =', graph.nodes[name]['url'])
    print(graph.nodes[name]['description'])
    inst = graph.nodes[name]['instanceof'] 
    if inst != []:
        instances = instances +inst
        ilist = [i[1] for i in inst]
        print('is an instance of ', ilist)

In [ ]:
showEntity(G, 'Cretaceous')

Cretaceous  wikiID= Q44626 wikipedia url = https://en.wikipedia.org/wiki/Cretaceous
"third and last period of the Mesozoic Era"
is an instance of  ['geological period']


In [ ]:
showEntity(G, 'geological period')


geological period  wikiID= Q392928 wikipedia url = notknown
"subdivision of geological time; shorter than an era and longer than an epoch"


In [ ]:
climate = set(['climate', 'climate-change'])
extinction =  set(['late-devonian', 'extinction', "climate-extinctions"])
human_extinct = set(['human-extinction'])
relativity = set(['relativity'])
blackholes = set(['relativity','blackhole','blackhole-neutron', 'gravitational-wave'])
qgrav = set(['relativity', 'quantum-gravity', 'quantum-grav3', 'string-theory'])
cosmo = set(['relativity', 'cosmology'])

def scorekey(key1, key2):
    if key1 in climate and key2 in climate:
        return 1
    if key1 in extinction and key2 in extinction:
        return 1
    if key1 in human_extinct and key2 in human_extinct:
        return 1
    if key1 in relativity and key2 in relativity:
        return 1
    if key1 in blackholes and key2 in blackholes:
        return 1
    if key1 in qgrav and key2 in qgrav:
        return 1
    if key1 in cosmo and key2 in cosmo:
        return 1
    return 0

In [ ]:
instances = []
classes = []
lines = []
ent_cnt = 0
for nd in G.nodes:
    #print(G.nodes[nd])
    node = G.nodes[nd]
    if node['flavor'] == 'entity':
        ent_cnt +=1
        inst = node['instanceof']
        instances = instances + inst
        clas = node['subclassof']
        classes = classes + clas
    elif node['flavor'] == 'article':
        #print('doing art ',nd, node)
        lines.append([node['context'], nd])
instanceSet = set(instances)
instances = list(instanceSet)
classSet = set(classes)
classes = list(classSet)
print('instances =', len(instances))
print("---------------")
print('classes =', len(classes))
print('lines =', len(lines))
print('entities = ', ent_cnt)

instances = 90
---------------
classes = 0
lines = 427
entities =  691


In [ ]:
sentences = [lines[i][0] for i in range(len(lines))]

In [ ]:
keys = [lines[i][1] for i in range(len(lines))]

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.8 MB/s 
     |████████████████████████████████| 4.4 MB 54.4 MB/s 
     |████████████████████████████████| 1.2 MB 58.2 MB/s 
     |████████████████████████████████| 101 kB 13.0 MB/s 
     |████████████████████████████████| 596 kB 57.6 MB/s 
     |████████████████████████████████| 6.6 MB 40.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=da1c31a3f45e919e2abfc8b35849a0707e171138b1d1d4093ca7b7952006894c
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
sentence_embeddings = sbert_model.encode(sentences)
import numpy as np
mar = np.zeros((len(sentences), 768))
for i in range(len(sentences)):
    x = np.linalg.norm(sentence_embeddings[i])
    mar[i] = sentence_embeddings[i]/x

In [ ]:
def find_in_keys(nodeid):
    for i in range(len(keys)):
        if nodeid == keys[i]:
            return i
    return -1

In [ ]:
def find_best(k, abstract, show=True):
    v = sbert_model.encode([abstract])[0]
    v0 = v/np.linalg.norm(v)
    norms = []
    for i in range(mar.shape[0]):
        norms.append([np.dot(v0,mar[i]), i])
    norms.sort(reverse=True)
    if show:
        print('top ',k, ' related nodes' )
    scor = 0.0
    #print(norms[0][1])
    nodename = keys[norms[0][1]]
    category =  G.nodes[nodename]['source']
    for i in range(k):
        node = keys[norms[i][1]]
        if scorekey(G.nodes[node]['source'],category) == 1:
            scor +=1.0
        if show:
            print(node)
            print(sentences[norms[i][1]])
    scor = scor/k
    return scor, norms[0:k]

In [ ]:
def find_best3(k, abstract, show=True):
    v = sbert_model.encode([abstract])[0]
    v0 = v/np.linalg.norm(v)
    norms = []
    for i in range(mar.shape[0]):
        norms.append([np.dot(v0,mar[i]), i])
    norms.sort(reverse=True)
    nodename = keys[norms[0][1]]
    newlist = []
    nopath = 0
    for i in range(40):
        #print(norms[i])
        x = norms[i][1]
        try:
            path = nx.shortest_path(G, nodename, 'Art'+str(x))
            #print(path)
            newlist.append(norms[i])
        except:
            nopath += 1
    norms = newlist
    ln = len(norms)
    if show:
        print('top ',k, ' related nodes' )
        if ln < k:
            print('but only found ', ln)
    scor = 0.0
    #print(norms[0][1])
    nodename = keys[norms[0][1]]
    category =  G.nodes[nodename]['source']
    for i in range(min(ln, k)):
        node = keys[norms[i][1]]
        if scorekey(G.nodes[node]['source'],category) == 1:
            scor +=1.0
        if show:
            print(node)
            print(sentences[norms[i][1]])
    scor = scor/min(ln, k)
    return scor, min(ln, k)

In [ ]:
find_best3(5, 'Black Holes are hide behind Event horizons.')

top  5  related nodes
Art0
The Theory of General Relativity demonstrates that Black Holes are hidden by event horizons.
Art61
Finkelstein's solution extended the Schwarzschild solution for the future of observers falling into a black hole.
Art76
Such "black neutron stars" were not thought possible and will mean ideas for how neutron stars and black holes form will need to be rethought.
Art34
In many ways, a black hole acts like an ideal black body, as it reflects no light.
Art108
â€œSo, the particle would actually smear across space as it falls, which would violate the equivalence principle".


(1.0, 5)

In [ ]:
def findClue(G, sent, show = True):
    cls = entityExtractorBlock(sent)
    clsent = [x[0] for x in cls]
    if show:
        print(clsent)
    arts = []
    for x in cls:
        item = x[0]
        for edg in G.edges():
            #print(edg)
            if edg[0] == item and G.nodes[edg[1]]['flavor'] == 'article':
                arts.append(edg[1])
    artset = set(arts)
    artnames = list(artset)
    factoredartnames = []
    for art in artnames:
        #context = G.nodes[art]['context']
        #print('context=',context)
        cls1ent = G.nodes[art]['ent-list']
        #cls1ent = [x[0] for x in cls1]
        cls1int = set(clsent).intersection(set(cls1ent))
        if cls1int != set():
            factoredartnames.append((art, len(cls1int)))
        artnames = [x[0] for x in factoredartnames]
    #if show:
    #    print('artnames=',artnames)
    dic = {}
    for x in artnames:
        dic[x] = 0
    for x in factoredartnames:
        dic[x[0]] = x[1]
    #print('DIC =',dic)
    bmax = 0
    bestmax = ''
    for x in artnames:
        if dic[x] > bmax:
            bmax = dic[x]
            #print(x)
            bestmax = x
    count_ties = 0
    tie_list = []
    for x in artnames:
        if dic[x] == bmax:
            count_ties += 1
            tie_list.append(x)
    if show:
        print(count_ties, 'nodes out of',len(artnames), 'matched the max value of ', bmax)
    if count_ties >  0.5*len(artnames) and len(artnames)> 2:
        return  []
    return tie_list
   

def findNeighbors(g, art):
    n = g.nodes[art]
    #print(n)
    features = []
    neighbors = []
    for x in g.edges:
        if x[1] == art:
            #print('found art in ',x)
            p = x[0]
            features.append(p)
            if p != 'Albert_Einstein' and p != 'Earth':
                for y in g.edges:
                    if y[0] == p:
                        if y[1] != art:
                            #print(y[1][0:3])
                            if y[1][0:3] != 'Art':
                                features.append(y[1])
                            else:
                                neighbors.append(y[1])
    return neighbors, features

mar2 = np.zeros((len(sentences), 768))
def computeMar2(G, mar, lambd):
    num_zero_neighbors = 0
    tot_neighbors = 0
    num_arts = 0
    for art in G.nodes():
        if G.nodes[art]['flavor'] == 'article':
            num_arts += 1.0
            neighbors, features =  findNeighbors(G, art)
            v = np.zeros((768))
            c = 0.0
            if len(neighbors) == 0:
                num_zero_neighbors +=1
            tot_neighbors += len(neighbors)
            if len(neighbors) > 0:
                for x in neighbors:
                    loc = find_in_keys(x)
                    v = v + mar[loc]
                    c = c+1.0
                v = v/c
                z = lambd*mar[find_in_keys(art)] + (1.0-lambd)*v
                z = z/np.linalg.norm(z)
                mar2[find_in_keys(art)] = z
            else:
                mar2[find_in_keys(art)] = mar[find_in_keys(art)]

    return num_zero_neighbors, tot_neighbors/num_arts


def find_best2(k, abstract, show=True):
    cls = findClue(G, abstract, show)
    #print('clues are =', cls)
    if cls != []:
        best = cls[0]
    else:
        _, x = find_best(1, abstract, show=False)
        best = keys[x[0][1]]
    if(len(cls)> 1):
        scora, normsa = find_best_wl2(2, cls[0], show)
        scorb, normsb = find_best_wl2(k-2, cls[1], show)
        scor = (2*scora+(k-2)*scorb)/k
        norms = normsa+normsb
    else:
        scor, norms = find_best_wl2(k, best, show)
    if show:
        print('score =', scor)
    return scor, norms


def find_best_wl2(k, nodeid, show=True):
    #print(nodeid)
    v0 = mar2[find_in_keys(nodeid)]
    norms = []
    for i in range(mar2.shape[0]):
        norms.append([np.dot(v0,mar2[i]), i])
    norms.sort(reverse=True)
    score = 0.0
    #if show:
    #    print('top ',k, ' related nodes' )
    scor = 0.0
    nodename = keys[norms[0][1]]
    category =  G.nodes[nodename]['source']
    for i in range(k):
        node = keys[norms[i][1]]
        sent = sentences[norms[i][1]]
        #print('node = ', node)
        nodea = node
        if scorekey(G.nodes[nodea]['source'],category) == 1:
            scor +=1.0
        if show:
            print(node)
            print(sent)
    scor = scor/k
    return scor, norms[0:k]
            
def showRelated(G,art):
    print( G.nodes[art]['context'])
    neighbors, features =  findNeighbors(G, art)
    n = set(neighbors)
    neighbors = list(n)
    print(neighbors)
    for x in neighbors:
        print( G.nodes[x]['context'])
        v0 = mar[find_in_keys(art)]
        v1 = mar[find_in_keys(x)]
        print(" dot prod =", np.dot(v0,v1))
    print(features)

In [ ]:
computeMar2(G, mar, 0.8)

(140, 5.02576112412178)

In [ ]:
def find_best2_5(k, text, show = True):
    _, norms = find_best2(40, text, show = False)
    nodename = keys[norms[0][1]]
    newlist = []
    nopath = 0
    for i in range(40):
        #print(norms[i])
        x = norms[i][1]
        try:
            path = nx.shortest_path(G, nodename, 'Art'+str(x))
            #print(path)
            newlist.append(norms[i])
        except:
            nopath += 1
    norms = newlist
    ln = len(norms)

    if show:
        print('top ',k, ' related nodes' )
    scor = 0.0
    #print(norms[0][1])
    nodename = keys[norms[0][1]]
    category =  G.nodes[nodename]['source']
    for i in range(min(k,ln)):
        node = keys[norms[i][1]]
        if scorekey(G.nodes[node]['source'],category) == 1:
            scor +=1.0
        if show:
            print(node)
            print(sentences[norms[i][1]])
            
    scor = scor/min(ln, k)
    return scor, min(ln,k)

In [ ]:
find_best2(4, 'Black Holes are hidden by an Event horizon.')

KeyboardInterrupt: ignored

In [ ]:
find_best(4, 'What is dark energy?')

top  4  related nodes
Art6
The Einstein field equations are nonlinear and very difficult to solve.
Art0
The Theory of General Relativity demonstrates that Black Holes are hidden by event horizons.
Art103
But, given that the two theories are so different, it is not clear that the idea still holds at the quantum level.
Art125
If one attempts to treat gravity as simply another quantum field, the resulting theory is not renormalizable.


(1.0,
 [[0.6555332870800294, 6],
  [0.646909281039126, 0],
  [0.6299279786147585, 103],
  [0.6216818531062869, 125]])

In [ ]:
find_best2(4, 'What is dark energy?')

KeyboardInterrupt: ignored

In [ ]:
find_best2(4, 'unification of general relativity and quantum theory is  supersymmetry known as supergravity')

In [ ]:
find_best(4, 'unification of general relativity and quantum theory is  supersymmetry known as supergravity')

top  4  related nodes
Art131
In what is called the second superstring revolution, it was conjectured that both string theory and a unification of general relativity and supersymmetry known as supergravity form part of a hypothesized eleven-dimensional model known as M-theory, which would constitute a uniquely defined and consistent theory of quantum gravity.
Art128
The central idea of string theory is to replace the classical concept of a point particle in quantum field theory, with a quantum theory of one-dimensional extended objects: string theory.
Art105
To this they added the principle of quantum superposition, which holds that particles can be smeared into more than one state at once.
Art112
Instead, Zych imagined kind of quantum conjoined twins, built from the quantum superposition of two different energy states - and therefore two superposed masses.


(1.0,
 [[0.8305973870672048, 131],
  [0.826438213203384, 128],
  [0.806596766876897, 105],
  [0.8039374808898, 112]])

In [ ]:
find_best(4, 'What is M-theory and what does it have to do with string theory and supersymmetry theory')

top  4  related nodes
Art128
The central idea of string theory is to replace the classical concept of a point particle in quantum field theory, with a quantum theory of one-dimensional extended objects: string theory.
Art131
In what is called the second superstring revolution, it was conjectured that both string theory and a unification of general relativity and supersymmetry known as supergravity form part of a hypothesized eleven-dimensional model known as M-theory, which would constitute a uniquely defined and consistent theory of quantum gravity.
Art120
What verifiable predictions does any theory of quantum gravity make?
Art119
How can the theory of quantum mechanics be merged with the theory of general relativity  gravitational force and remain correct at microscopic length scales?


(1.0,
 [[0.8063263416009451, 128],
  [0.7838318447451832, 131],
  [0.7744987105380479, 120],
  [0.7575366256356751, 119]])

In [ ]:
find_best2(4, 'What is M-theory and what does it have to do with string theory and supersymmetry theory')

KeyboardInterrupt: ignored

In [ ]:
artlist = []
for n in G.nodes():
    if G.nodes[n]['flavor'] == 'article':
        artlist.append(n)

In [ ]:
numarts = len(artlist)
print(numarts)
scor = 0
for n in artlist:
    doc = G.nodes[n]['context']
    s, _ = find_best(4, doc, show=False)
    scor += s
print(scor/numarts)

427
0.6984777517564403


In [ ]:
numarts = len(artlist)
print(numarts)
scor = 0
num_retured = 0
for n in artlist:
    doc = G.nodes[n]['context']
    #print(doc)
    #s, _ = find_best(4, doc[0], show=False)
    s,  x = find_best3(4, doc, show=False)
    scor += s
    num_retured += x
print(scor/numarts, num_retured/numarts)
    

427
0.801912568306011 3.2365339578454333


In [ ]:
numarts = len(artlist)
print(numarts)
scor = 0
for n in artlist:
    doc = G.nodes[n]['context']
    s, _ = find_best2(4, doc, show=False)
    scor += s
print(scor/numarts)

KeyboardInterrupt: ignored

In [ ]:
numarts = len(artlist)
print(numarts)
scor = 0
num_returned = 0
for n in artlist:
    doc = G.nodes[n]['context']
    s, x = find_best2_5(4, doc, show=False)
    scor += s
    num_returned +=x
print(scor/numarts, num_returned/numarts)


In [ ]:
def computeMar3(G, mar, lambd):
    num_zero_neighbors = 0
    tot_neighbors = 0
    num_arts = 0
    for art in G.nodes():
        if G.nodes[art]['flavor'] == 'article':
            num_arts += 1.0
            neighbors, features =  findNeighbors(G, art)
            v = np.zeros((768))
            c = 0.0
            if len(neighbors) == 0:
                num_zero_neighbors +=1
            tot_neighbors += len(neighbors)
            if len(neighbors) > 0:
                for x in neighbors:
                    loc = find_in_keys(x)
                    v = v + mar[loc]
                    c = c+1.0
                v = v/c
                z = lambd*mar[find_in_keys(art)] + (1.0-lambd)*v
                z = z/np.linalg.norm(z)
                mar3[find_in_keys(art)] = z
            else:
                mar3[find_in_keys(art)] = mar[find_in_keys(art)]

    return num_zero_neighbors, tot_neighbors/num_arts

In [ ]:
mar3 = np.zeros((len(sentences), 768))
computeMar3(G, mar2,0.8)
mar2 = mar3
numarts = len(artlist)
print(numarts)
scor = 0
for n in artlist:
    s, _ = find_best2(4, n, show=False)
    scor += s
print(scor/numarts)

427


RetryError: ignored

In [ ]:
mar3 = np.zeros((len(sentences), 768))
computeMar3(G, mar2,0.8)
mar2 = mar3
numarts = len(artlist)
print(numarts)
scor = 0
for n in artlist:
    s, _ = find_best_wl2(4, n, show=False)
    scor += s
print(scor/numarts)

In [ ]:
def remove_trailing_and(text):
    i = text.find('and')
    if i >0:
        i = text.rindex('and')
        if len(text)-i < 7:
            return text[0:i]
        else:
            return text
    else:
        return text
    
def explain(sg, entity):
    n = sg.nodes[entity]
    inst = n['instanceof']
    instancefound = False
    sent =  entity 
    #print('=======================================')
    if n['description'] != '':
        preline = entity+'is defined as ', n['description']
    else: 
        preline = ''
    if inst != []:
        instancefound = True
        sent = sent +' is a '
        cnt = 0
        for ins in inst:
            endline = ''
            if cnt < len(inst)-1:
                endline = ' and '
                cnt = cnt+1
            if (ins[1].find('organism') < 0) and (ins[1].find('first-order') < 0) :
                sent = sent+ ins[1]+ endline
            else:
                cnt = cnt-1
    sent =  remove_trailing_and(sent)      
    inst = n['subclassof']
    if inst != []:
        if instancefound == True:
            sent = sent + ' and '
        else:
            sent = sent # + ' is '
        #sent = sent + ' a subclass of '
        #for ins in inst:
        #    sent = sent+ ins[1]+' and '
   
    print(preline, remove_trailing_and(sent))

In [50]:
def makeArtSubGraph(G, artlist, do_closure = True):
    oe = G.edges
    sgl = artlist
    # compute links from articles to entities
    sgl2 = []
    for x in oe:
        if x[0] in set(sgl):
            sgl2.append(x[1])
    sgl = sgl + sgl2
    sgl = set(sgl)
    sgl = list(sgl)
    #add links from entities and articles to other things
    sgl2 = []
    for x in oe:
        if x[1] in set(sgl):
            sgl2.append(x[0])
    sgl = sgl + sgl2
    sgl = set(sgl)
    sgl = list(sgl)
    
    if do_closure:
        sgl2 = []
        for x in oe:
            if x[0] in set(sgl):
                sgl2.append(x[1])
        sgl = sgl + sgl2
        sgl = set(sgl)
        sgl = list(sgl)

    sglab = {}
    for x in sgl:
        sglab[x] = x
    sg = nx.subgraph(G, sgl)
    return sglab, sg

In [51]:
def buildMatchingGraph(G, text , do_closure = True):
    _, s = find_best2(5, text, show=False)
    print('s=',s)
    artlist = [keys[it[1]] for it in s]
    print(len(artlist))
    for state in artlist:
        print(state+'--', G.nodes[state]['context'])
    _, sg = makeArtSubGraph(G, artlist, do_closure)
    showGraph(sg)
    print("related entities are:")
    print('=====================================')
    for nd in sg:
        n = sg.nodes[nd]
        if n['flavor']== 'entity':
            explain(sg, nd)
            print(" ")
    return sg

In [52]:
def explainPath(cop, X, Y):
    path = nx.shortest_path(cop, X, Y)
    print(path)
    for a in path:
        print("----------------")
        if cop.nodes[a]['flavor'] == "entity":
            print(a, cop.nodes[a]['description'])
        if cop.nodes[a]['flavor'] == 'article':
            print(a, cop.nodes[a]['context'])

In [69]:
sg = buildMatchingGraph(G,'The central idea of string theory is to replace the classical concept of a point particle in quantum field theory, with a quantum theory of one-dimensional extended objects: string theory.')

RetryError: ignored

In [55]:
explainPath(sg,  'Art273', 'Yucatán_Peninsula')

NameError: ignored

In [56]:
numarts = len(artlist)
print(numarts)
scor = 0
neighbors = np.zeros(5)
for n in artlist:
    nbrs, _ = findNeighbors(G, n)
    l = len(nbrs)
    if l > 4:
        l = 4
    neighbors[l]+=1
print(neighbors)

427
[140.  57.  37.  26. 167.]


In [57]:
tot = 0
for i in range(5):
    tot += neighbors[i]*i
tot/numarts

2.053864168618267

In [58]:
numarts = len(artlist)
print(numarts)
scor = 0
neighbors = np.zeros(5)
for n in artlist:
    nbrs, _ = findNeighbors(G, n)
    dubnbrs = []
    for x in nbrs:
        if x != n:
            dub, _ = findNeighbors(G, x)
            dubnbrs = dubnbrs + dub
    sdubnbrs = set(dubnbrs)
    nbrs = list(sdubnbrs)
    l = len(nbrs)
    if l > 4:
        l = 4
    neighbors[l]+=1
print(neighbors)

427
[140.  32.  12.   8. 235.]


In [59]:
tot = 0
for i in range(5):
    tot += neighbors[i]*i
tot/numarts

2.388758782201405

In [60]:
140/427

0.32786885245901637

In [61]:
comp = nx.connected_components(G)

In [62]:
num_comp = 0
comp_size = np.zeros(7)
for x in comp:
    x_size =  0
    for n in x:
        if G.node[n]['flavor'] == 'article':
            x_size += 1
    if x_size > 6:
        comp_size[6] = x_size
    else:
        comp_size[x_size] += 1
print(comp_size)

AttributeError: ignored

In [63]:
total_nodes = 0
for i in range(6):
    total_nodes += comp_size[i]*i
total_nodes + 304

304.0

In [64]:
tot = (76+10*2 +3*3 +4*(2+2+304))/427
tot

3.1311475409836067